# Movie Lens dataset

## Extract a recommended sub-dataset
Here, we extract the 25M dataset.

In [1]:
import tensorflow_datasets as tfds

# Load the dataset
dataset = tfds.load('movielens/25m-ratings', split='train')

2024-07-08 08:37:17.169485: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-08 08:37:17.195173: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 08:37:17.868147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-08 08:37:18.681367: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-

In [2]:
# Display the first 10 observations
for i, data in enumerate(dataset.take(10)):
    print(data)

{'movie_genres': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([4, 5])>, 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'7367'>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Ladykillers, The (2004)'>, 'timestamp': <tf.Tensor: shape=(), dtype=int64, numpy=1198090049>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'58198'>, 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=3.5>}
{'movie_genres': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([15, 11])>, 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'109487'>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Interstellar (2014)'>, 'timestamp': <tf.Tensor: shape=(), dtype=int64, numpy=1438858445>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'149681'>, 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=4.0>}
{'movie_genres': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 5,  7, 13, 14, 16])>, 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'6885'>, 'mo

2024-07-08 08:37:19.275446: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [3]:
import csv
from tqdm import tqdm

# Define the file path
csv_file = 'Recommended/dataset.csv'

# Open the CSV file in write mode
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the column names to the CSV file
    writer.writerow(['Movie Index', 'User Index', 'Rating'])

    # Loop through the dataset and write the data to the CSV file with progress bar
    for data in tqdm(dataset, total=len(dataset), desc="Writing data"):
        movie_id = data['movie_id'].numpy().decode()
        user_id = data['user_id'].numpy().decode()
        user_rating = data['user_rating'].numpy()
        writer.writerow([movie_id, user_id, user_rating])

print("Data has been written to", csv_file)

Writing data:   0%|          | 0/25000095 [00:00<?, ?it/s]

Writing data: 100%|██████████| 25000095/25000095 [45:08<00:00, 9231.34it/s]

Data has been written to Recommended/dataset.csv


In [4]:
import pandas as pd

# Read the CSV dataset file
df = pd.read_csv('Recommended/dataset.csv')

# Remap user ids
df['User Index'] = df['User Index'].rank(method='dense') - 1

# Remap movie ids
df['Movie Index'] = df['Movie Index'].rank(method='dense') - 1

# Convert the columns to integers
df['User Index'] = df['User Index'].astype(int)
df['Movie Index'] = df['Movie Index'].astype(int)

# Write the updated dataset to the CSV file
df.to_csv('Recommended/dataset.csv', index=False)

In [6]:
# Read the CSV dataset file
df = pd.read_csv('Recommended/dataset.csv')

# Get the range of movie IDs and user IDs
min_movie_id = df['Movie Index'].min()
max_movie_id = df['Movie Index'].max()
min_user_id = df['User Index'].min()
max_user_id = df['User Index'].max()

# Get the total number of unique movie IDs and user IDs
unique_movie_ids = df['Movie Index'].nunique()
unique_user_ids = df['User Index'].nunique()

# Verify the range of movie IDs and user IDs
print("Movie ID range:", min_movie_id, "-", max_movie_id)
print("User ID range:", min_user_id, "-", max_user_id)
print("Unique Movie IDs:", unique_movie_ids)
print("Unique User IDs:", unique_user_ids)

Movie ID range: 0 - 59046
User ID range: 0 - 162540
Unique Movie IDs: 59047
Unique User IDs: 162541


## For small dataset

In [7]:
import tensorflow_datasets as tfds

# Load the dataset
dataset = tfds.load('movielens/100k-ratings', split='train')

In [8]:
import pandas as pd

# Create a pandas dataframe
df = tfds.as_dataframe(dataset)

# Extract the user id, movie id, and rating
df = df[['movie_id', 'user_id', 'user_rating']]
df['movie_id'] = pd.to_numeric(df['movie_id'])
df['movie_id'] = df['movie_id'] - 1
df['user_id'] = pd.to_numeric(df['user_id'])
df['user_id'] = df['user_id'] - 1
df.rename(columns={'user_rating': 'Rating', 'movie_id': 'Movie Index', 'user_id': 'User Index'}, inplace=True)

# Display the dataframe
df.head()

2024-07-08 09:23:11.250857: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/tmp/ipykernel_6929/2888648731.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['movie_id'] = pd.to_numeric(df['movie_id'])
/tmp/ipykernel_6929/2888648731.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['movie_id'] = df['movie_id'] - 1
/tmp/ipykernel_6929/2888648731.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,Movie Index,User Index,Rating
0,356,137,4.0
1,708,91,2.0
2,411,300,4.0
3,55,59,4.0
4,894,196,3.0


In [9]:
# Remap user ids
df['User Index'] = df['User Index'].rank(method='dense') - 1

# Remap movie ids
df['Movie Index'] = df['Movie Index'].rank(method='dense') - 1

# Convert the columns to integers
df['User Index'] = df['User Index'].astype(int)
df['Movie Index'] = df['Movie Index'].astype(int)

/tmp/ipykernel_6929/1170244081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['User Index'] = df['User Index'].rank(method='dense') - 1
/tmp/ipykernel_6929/1170244081.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Movie Index'] = df['Movie Index'].rank(method='dense') - 1
/tmp/ipykernel_6929/1170244081.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [10]:
unique_movie_ids = df['Movie Index'].nunique()
unique_user_ids = df['User Index'].nunique()
density = len(df) / (unique_movie_ids * unique_user_ids)

print("Number of unique movie ids:", unique_movie_ids)
print("Number of unique user ids:", unique_user_ids)
print("Density of the dataset:", density)

Number of unique movie ids: 1682
Number of unique user ids: 943
Density of the dataset: 0.06304669364224531


In [11]:
df.to_csv('Small/dataset.csv', index=False)

In [12]:
import pandas as pd

# Read the CSV dataset file
df = pd.read_csv('Small/dataset.csv')

# Get the range of movie IDs and user IDs
min_movie_id = df['Movie Index'].min()
max_movie_id = df['Movie Index'].max()
min_user_id = df['User Index'].min()
max_user_id = df['User Index'].max()

# Get the total number of unique movie IDs and user IDs
unique_movie_ids = df['Movie Index'].nunique()
unique_user_ids = df['User Index'].nunique()

# Verify the range of movie IDs and user IDs
print("Movie ID range:", min_movie_id, "-", max_movie_id)
print("User ID range:", min_user_id, "-", max_user_id)
print("Unique Movie IDs:", unique_movie_ids)
print("Unique User IDs:", unique_user_ids)

Movie ID range: 0 - 1681
User ID range: 0 - 942
Unique Movie IDs: 1682
Unique User IDs: 943
